# Imports

In [1]:
import csv
import os
from utils import get_id_list, get_data, write_file

from dotenv import load_dotenv
load_dotenv()

True

# Grabbing All Movie IDs

In [2]:
TMBD_API_KEY = os.getenv('TMBD_API_KEY')
YEARS = list(range(2019, 2023))
CSV_HEADER = ['Title', 'Runtime (minutes)', 'Language', 'Overview',
              'Release Date', 'Genre', 'Keywords',
              'Actors', 'Directors', 'Stream', 'Buy', 'Rent', 
              'Production Companies', 'Website']

# Writing all IDs to CSV file

In [3]:
for year in YEARS:
    # Grab list of ids for all films made in {YEAR}
    movie_list = list(set(get_id_list(TMBD_API_KEY, year)))
    print(movie_list)

    FILE_NAME = f'./data/{year}_movie_collection_data.csv'

    # Creating file
    with open(FILE_NAME, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(CSV_HEADER)

    # Iterate through list of ids to get data
    for id in movie_list:
        dict = get_data(TMBD_API_KEY, id)
        write_file(FILE_NAME, dict)

['492188', '290859', '456740', '499338', '450465', '423204', '512200', '519465', '517093', '481084', '571891', '454640', '568160', '592350', '535167', '565373', '533642', '484641', '530915', '474350', '453405', '438650', '491854', '329996', '554596', '320288', '501170', '480414', '506574', '429617', '420818', '537061', '549945', '512895', '181812', '575222', '645488', '399361', '330457', '530385', '546554', '331482', '299537', '301528', '570269', '338967', '517764', '458305', '567609', '473033', '522938', '479455', '700348', '431693', '398978', '384018', '581387', '515001', '475557', '420809', '531309', '572154', '487297', '509967', '820232', '638507', '527641', '461130', '512196', '420817', '411728', '496243', '399579', '514439', '540901', '166428', '518596', '581528', '619264', '447404', '287947', '373571', '575299', '610892', '521029', '595172', '532067', '506517', '466272', '412117', '299534', '586347', '522924', '554600', '559969', '537915', '486589', '458156', '522518', '359724']

# Random Testing

In [4]:
import requests
import time
import json
from iso639 import languages

max_retries = 5

# Test the function
Movie_ID = '545611'
# Create empty index
API_key = os.getenv('TMBD_API_KEY')
query = 'https://api.themoviedb.org/3/movie/' + Movie_ID + \
    '?api_key='+API_key +'&append_to_response=keywords,' + \
        'watch/providers,credits&language=en-US'

response = requests.get(query)
for i in range(max_retries):
    if response.status_code == 429:
        # If the response was a 429, wait and then try again
        print(
            f"Request limit reached. Waiting and retrying ({i+1}/{
                max_retries})")
        time.sleep(2 ** i)  # Exponential backoff
    else:
        dict = response.json()

# Pretty print the JSON data
pretty_json = json.dumps(dict, indent=4)

# print(pretty_json)

title = dict['title']
runtime = str(dict['runtime']) + " minutes"
language_code = dict['original_language']
release_date = dict['release_date']
overview = dict['overview']
all_genres = dict['genres']
website = 'movie_collection_data.csv' if dict['homepage'] == '' else dict['homepage']
prod_companies = dict['production_companies']

# Converting language
if language_code != 'xx':
    language = languages.get(alpha2=language_code).name
else:
    language = 'Unknown'

# Parsing genres
genre_str = ""
for genre in all_genres:
    genre_str += genre['name'] + ", "
genre_str = genre_str[:-2]

# Parsing keywords
all_keywords = dict['keywords']['keywords']
keyword_str = ""
for keyword in all_keywords:
    if is_english(keyword['name']):
        keyword_str += keyword['name'] + ", "
if keyword_str == "":
    keyword_str = "None" 
else:
    keyword_str = keyword_str[:-2]
print(keyword_str)

# Parsing watch providers
watch_providers = dict['watch/providers']['results']
stream_str, buy_str, rent_str = "", "", ""
if 'US' in watch_providers:
    watch_providers = watch_providers['US']
    provider_strings = ['flatrate', 'buy', 'rent']
    for string in provider_strings:
        if string not in watch_providers:
            continue

        _str = ""

        for element in watch_providers[string]:
            _str += element['provider_name'] + ", "
        _str = _str[:-2] + " "

        if string == 'flatrate':
            stream_str += _str
        elif string == 'buy':
            buy_str += _str
        else:
            rent_str += _str


credits = dict['credits']
actor_list, director_list = [], []

# Parsing cast
cast = credits['cast']
NUM_ACTORS = 5
for member in cast[:NUM_ACTORS]:
    actor_list.append(member["name"])

# Parsing crew
crew = credits['crew']
for member in crew:
    if member['job'] == 'Director':
        director_list.append(member["name"])

actor_str = ', '.join(list(set(actor_list)))
director_str = ', '.join(list(set(director_list)))

print(f"Actors: {actor_str}")
print(f"Director: {director_str}")

# Parsing production companies
prod_str = ""
for company in prod_companies:
    prod_str += company['name'] + ", "
prod_str = prod_str[:-2]
print(prod_str)

NameError: name 'is_english' is not defined